# ipyparallel with simple MNIST classifier
 - sample from here: https://keras.io/examples/vision/mnist_convnet/

In [1]:
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers

In [2]:
# Model / data parameters
num_classes = 10
input_shape = (28, 28, 1)

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# Scale images to the [0, 1] range
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255
# Make sure images have shape (28, 28, 1)
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)
print("x_train shape:", x_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")


# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [3]:
model = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation="softmax"),
    ]
)

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
flatten (Flatten)            (None, 1600)              0         
_________________________________________________________________
dropout (Dropout)            (None, 1600)              0         
_________________________________________________________________
dense (Dense)                (None, 10)                1

In [4]:
batch_size = 128
epochs = 15

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)

Epoch 1/15
422/422 [==============================] - 11s 26ms/step - loss: 0.3666 - accuracy: 0.8897 - val_loss: 0.0862 - val_accuracy: 0.9752
Epoch 2/15
422/422 [==============================] - 11s 27ms/step - loss: 0.1095 - accuracy: 0.9670 - val_loss: 0.0597 - val_accuracy: 0.9827
Epoch 3/15
422/422 [==============================] - 11s 27ms/step - loss: 0.0816 - accuracy: 0.9754 - val_loss: 0.0459 - val_accuracy: 0.9875
Epoch 4/15
422/422 [==============================] - 11s 27ms/step - loss: 0.0677 - accuracy: 0.9788 - val_loss: 0.0411 - val_accuracy: 0.9892
Epoch 5/15
422/422 [==============================] - 11s 26ms/step - loss: 0.0588 - accuracy: 0.9822 - val_loss: 0.0353 - val_accuracy: 0.9898
Epoch 6/15
422/422 [==============================] - 11s 26ms/step - loss: 0.0547 - accuracy: 0.9833 - val_loss: 0.0338 - val_accuracy: 0.9902
Epoch 7/15
422/422 [==============================] - 11s 26ms/step - loss: 0.0496 - accuracy: 0.9844 - val_loss: 0.0319 - val_accuracy:

In [5]:
score = model.evaluate(x_test, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

Test loss: 0.022799959406256676
Test accuracy: 0.9927999973297119


In [6]:
model.save('simple_mnist.h5')

# setup ipyparallel engines and view 

In [7]:
import ipyparallel as ipp
import time

In [8]:
! ipcluster start -n 4 --daemonize

In [9]:
client = ipp.Client()

In [10]:
view = client[:]

## setup prediction data from test data 

In [11]:
x_test.shape

(10000, 28, 28, 1)

In [12]:
ds_timing = []

In [13]:
pred1 = x_test[0:1000]
pred2 = x_test[1000:2000]
pred3 = x_test[2000:3000]
pred4 = x_test[3000:4000]

lbl1 = y_test[0:1000]
lbl2 = y_test[1000:2000]
lbl3 = y_test[2000:3000]
lbl4 = y_test[3000:4000]

In [14]:
ds_timing.append((pred1, lbl1))
ds_timing.append((pred2, lbl2))
ds_timing.append((pred3, lbl3))
ds_timing.append((pred4, lbl4))

## sequential prediction timing 

In [15]:
for ds in ds_timing:
    pred, lbl = ds
    start = time.time()
    predictions = model.predict(pred)
    print(f'time: {time.time() - start} seconds')

time: 0.17849993705749512 seconds
time: 0.11699891090393066 seconds
time: 0.11700057983398438 seconds
time: 0.11749958992004395 seconds


In [16]:
start = time.time()
%time predictions = model.predict(pred1)
print(f'time: {time.time() - start} seconds')

Wall time: 115 ms
time: 0.11550164222717285 seconds


In [17]:
np.argmax(predictions, axis=1)

array([7, 2, 1, 0, 4, 1, 4, 9, 5, 9, 0, 6, 9, 0, 1, 5, 9, 7, 3, 4, 9, 6,
       6, 5, 4, 0, 7, 4, 0, 1, 3, 1, 3, 4, 7, 2, 7, 1, 2, 1, 1, 7, 4, 2,
       3, 5, 1, 2, 4, 4, 6, 3, 5, 5, 6, 0, 4, 1, 9, 5, 7, 8, 9, 3, 7, 4,
       6, 4, 3, 0, 7, 0, 2, 9, 1, 7, 3, 2, 9, 7, 7, 6, 2, 7, 8, 4, 7, 3,
       6, 1, 3, 6, 9, 3, 1, 4, 1, 7, 6, 9, 6, 0, 5, 4, 9, 9, 2, 1, 9, 4,
       8, 7, 3, 9, 7, 4, 4, 4, 9, 2, 5, 4, 7, 6, 7, 9, 0, 5, 8, 5, 6, 6,
       5, 7, 8, 1, 0, 1, 6, 4, 6, 7, 3, 1, 7, 1, 8, 2, 0, 2, 9, 9, 5, 5,
       1, 5, 6, 0, 3, 4, 4, 6, 5, 4, 6, 5, 4, 5, 1, 4, 4, 7, 2, 3, 2, 7,
       1, 8, 1, 8, 1, 8, 5, 0, 8, 9, 2, 5, 0, 1, 1, 1, 0, 9, 0, 3, 1, 6,
       4, 2, 3, 6, 1, 1, 1, 3, 9, 5, 2, 9, 4, 5, 9, 3, 9, 0, 3, 6, 5, 5,
       7, 2, 2, 7, 1, 2, 8, 4, 1, 7, 3, 3, 8, 8, 7, 9, 2, 2, 4, 1, 5, 9,
       8, 7, 2, 3, 0, 4, 4, 2, 4, 1, 9, 5, 7, 7, 2, 8, 2, 6, 8, 5, 7, 7,
       9, 1, 8, 1, 8, 0, 3, 0, 1, 9, 9, 4, 1, 8, 2, 1, 2, 9, 7, 5, 9, 2,
       6, 4, 1, 5, 8, 2, 9, 2, 0, 4, 0, 0, 2, 8, 4,

In [18]:
np.argmax(lbl1, axis=1)

array([7, 2, 1, 0, 4, 1, 4, 9, 5, 9, 0, 6, 9, 0, 1, 5, 9, 7, 3, 4, 9, 6,
       6, 5, 4, 0, 7, 4, 0, 1, 3, 1, 3, 4, 7, 2, 7, 1, 2, 1, 1, 7, 4, 2,
       3, 5, 1, 2, 4, 4, 6, 3, 5, 5, 6, 0, 4, 1, 9, 5, 7, 8, 9, 3, 7, 4,
       6, 4, 3, 0, 7, 0, 2, 9, 1, 7, 3, 2, 9, 7, 7, 6, 2, 7, 8, 4, 7, 3,
       6, 1, 3, 6, 9, 3, 1, 4, 1, 7, 6, 9, 6, 0, 5, 4, 9, 9, 2, 1, 9, 4,
       8, 7, 3, 9, 7, 4, 4, 4, 9, 2, 5, 4, 7, 6, 7, 9, 0, 5, 8, 5, 6, 6,
       5, 7, 8, 1, 0, 1, 6, 4, 6, 7, 3, 1, 7, 1, 8, 2, 0, 2, 9, 9, 5, 5,
       1, 5, 6, 0, 3, 4, 4, 6, 5, 4, 6, 5, 4, 5, 1, 4, 4, 7, 2, 3, 2, 7,
       1, 8, 1, 8, 1, 8, 5, 0, 8, 9, 2, 5, 0, 1, 1, 1, 0, 9, 0, 3, 1, 6,
       4, 2, 3, 6, 1, 1, 1, 3, 9, 5, 2, 9, 4, 5, 9, 3, 9, 0, 3, 6, 5, 5,
       7, 2, 2, 7, 1, 2, 8, 4, 1, 7, 3, 3, 8, 8, 7, 9, 2, 2, 4, 1, 5, 9,
       8, 7, 2, 3, 0, 4, 4, 2, 4, 1, 9, 5, 7, 7, 2, 8, 2, 6, 8, 5, 7, 7,
       9, 1, 8, 1, 8, 0, 3, 0, 1, 9, 9, 4, 1, 8, 2, 1, 2, 9, 7, 5, 9, 2,
       6, 4, 1, 5, 8, 2, 9, 2, 0, 4, 0, 0, 2, 8, 4,

## parallel prediction timing 

In [19]:
def p_prediction(ds):
    import time
    import tensorflow as tf
    model = tf.keras.models.load_model('simple_mnist.h5') # load pretrained model we saved earlier
    
    pred, lbl = ds
    start = time.time()
    predictions = model.predict(pred)
    prediction_time = time.time() - start
    
    return (prediction_time, predictions)

### map function above over the list of 4 datasets created earlier

In [20]:
p_results = view.map(p_prediction, ds_timing).get()

In [21]:
for res in p_results:
    p_time, pre = res
    print(p_time)

0.22250127792358398
0.22500061988830566
0.2245018482208252
0.24450135231018066


## kill cluster engines and client 

In [22]:
! ipcluster stop

client.shutdown()
client.close()

2021-05-02 19:12:23.381 [IPClusterStop] Removing pid file: C:\Users\ghastly\.ipython\profile_default\pid\ipcluster.pid
